In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Funcoes_auxiliares.func_aux import *
from sklearn.impute import KNNImputer

# Knowing the dataset
At first, no cleaning will be applied to the dataset, later we will apply all cleaning and make a new EDA

In [2]:
#Import dataset
df = pd.read_csv('base_suja/base_unificada_suja.csv')
variaveis = [
    'data_evento', 'ano_evento', 'TIPOBITO', 'evento_MUNNOMEX',
    'res_MUNNOMEX', 'evento_CAPITAL', 'res_CAPITAL', 'evento_REGIAO',
    'res_REGIAO', 'evento_SIGLA_UF', 'res_SIGLA_UF', 'IDADEMAE',
    'idademae_faixa', 'ESCMAE2010', 'escolaridade_mae', 'OBITOGRAV',
    'GRAVIDEZ', 'tipo_gravidez', 'SEMAGESTAC', 'idade_gestacao_faixa',
    'SEXO', 'def_sexo', 'PESO', 'peso_faixa', 'OBITOPARTO',
    'def_obito_parto', 'CAUSABAS', 'causabas_capitulo',
    'causabas_categoria', 'causabas_grupo', 'causabas_subcategoria',
    'FLAG_BASE', 'sum_CENTROBS', 'sum_QTINST34', 'sum_QTINST35',
    'sum_QTINST36', 'sum_QTINST37', 'sum_QTLEIT34', 'sum_QTLEIT38',
    'sum_QTLEIT39', 'sum_QTLEIT40', 'sum_CENTRNEO','TP_UNID_5', 'TP_UNID_7',
    'TP_UNID_15', 'TP_UNID_36', 'TP_UNID_61'
]
df = df[variaveis]

<ipython-input-2-5c667751b380>:2: DtypeWarning: Columns (25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('base_suja/base_unificada_suja.csv')


In [ ]:
# Configuration to show more rows and columns
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
url_dicionario = 'https://docs.google.com/spreadsheets/d/1QFy_F2o81ULglNx8knNqg6oI7v3RARaUAaLLwOQr7K4/edit?usp=sharing'
f"""The dataset has {len(df)} lines and {len(df.columns)} columns. More details can be consulted in the dictionary {url_dicionario}. Below the type of variables and the 'face' from the dataset"""

In [ ]:
df.info()

In [ ]:
df.head()

#  Checking data quality

### Duplicates

In [ ]:
# Duplicate data 
duplicados = df.duplicated()
soma = duplicados.sum()
f"""The dataset has {soma} duplicate rows, which represents {round((soma/len(df)) * 100, 2)} %"""

In [ ]:
# Frequency of duplicates by FLAG_BASE, res_SIGLA_UF and ano_evento
df_duplicados = df[duplicados]
df_duplicados.value_counts(['FLAG_BASE', 'ano_evento', 'res_SIGLA_UF'])

# There is no concentration of missing items that indicates a structural problem with filling in the data, whether by year or 
# UF of residence

### Missing

In [ ]:
# Counting missing values
df_sim_dofet = df[df['FLAG_BASE']=='SIM_DOFET']
missing_count_sim_dofet = df_sim_dofet.isnull().sum()  # counts the null values in each column
missing_percent_sim_dofet = round((missing_count_sim_dofet / len(df_sim_dofet)) * 100,2)  # calculates the percentage of null values
missing_data_sim_dofet = pd.DataFrame({'Missing Count': missing_count_sim_dofet, 'Missing Percentage': missing_percent_sim_dofet})
missing_data_sim_dofet.index.name = 'Variable'  # sets the index name to 'Variable'
missing_data_sim_dofet.reset_index(inplace=True)  # resets the index to make 'Variable' a column
missing_data_sim_dofet['BASE'] = 'SIM_DOFET'

df_sinasc = df[df['FLAG_BASE']=='SINASC']
missing_count_sinasc = df_sinasc.isnull().sum()  # counts the null values in each column
missing_percent_sinasc = round((missing_count_sinasc / len(df_sinasc)) * 100,2)  # calculates the percentage of null values
missing_data_sinasc = pd.DataFrame({'Missing Count': missing_count_sinasc, 'Missing Percentage': missing_percent_sinasc})
missing_data_sinasc.index.name = 'Variable'  # sets the index name to 'Variable'
missing_data_sinasc.reset_index(inplace=True)  # resets the index to make 'Variable' a column
missing_data_sinasc['BASE'] = 'SINASC'

# appending
missing_data = pd.concat([missing_data_sim_dofet, missing_data_sinasc])

# Sorting the DataFrame by the highest missing frequencies
missing_data_sorted = missing_data.sort_values(by=['BASE', 'Missing Count'], ascending=False)

missing_data_sorted[missing_data_sorted['Missing Percentage'] > 0]

# Some variables are not filled in the sinasc dataset. For EDA it will be used in SIM_DOFET, but in the model it will not be removed
# 'OBITOGRAV' no padding at the base
# Missing points for other variables will be removed, keeping the variables in the study

In [ ]:
del missing_data_sorted, missing_data, df_sim_dofet, df_sinasc

In [ ]:
# Checking if there is any pattern in the missing data looking at year and 'res_SIGLA_UF'
lista = ['ESCMAE2010', 'SEMAGESTAC', 'PESO', 'IDADEMAE']
for i in lista:
    print(f"""Variável {i}\n\n{df.loc[df[i].isnull(), ['FLAG_BASE', 'ano_evento', 'res_SIGLA_UF']].value_counts()}""")

# Some UF's have very high values, but compared to the total UF it is less than 10%
# len(df.loc[(df['res_SIGLA_UF']=='CE') & (df['ano_evento']==2021) & (df['FLAG_BASE']=='SINASC')])

Variável IDADEMAE

FLAG_BASE  ano_evento  res_SIGLA_UF
SIM_DOFET  2021        SP              505
           2018        SP              474
           2020        SP              469
           2019        SP              400
           2022        SP              389
           2018        BA              251
           2020        BA              238
           2019        MG              233
           2020        MG              231
           2021        MG              231
           2018        MG              230
           2019        BA              225
           2021        BA              202
           2022        MG              198
                       BA              190
           2018        PE              174
                       RJ              158
           2019        RJ              157
                       PE              155
           2020        RJ              132
           2021        RJ              128
           2018        PA              112

### Ignorado

In [ ]:
df_ignorado = df.loc[(df['ESCMAE2010']==9) | (df['GRAVIDEZ']==9) |
                     (df['idade_gestacao_faixa']=='Ignorado') | (df['SEXO']==0)]
df_ignorado['FLAG_BASE'].value_counts()

### Categorical variables

In [ ]:
lista_cat = [
'ano_evento'
, 'TIPOBITO'       
, 'evento_MUNNOMEX'
, 'res_MUNNOMEX'
, 'evento_CAPITAL'
, 'res_CAPITAL'
, 'evento_REGIAO'
, 'res_REGIAO'
, 'evento_SIGLA_UF'
, 'res_SIGLA_UF'
, 'idademae_faixa'
, 'ESCMAE2010'
, 'escolaridade_mae'
, 'OBITOGRAV'
, 'GRAVIDEZ'
, 'tipo_gravidez'
, 'idade_gestacao_faixa'
, 'SEXO'
, 'def_sexo'
, 'peso_faixa'
, 'OBITOPARTO'
, 'def_obito_parto'
, 'CAUSABAS'
, 'causabas_capitulo'
, 'causabas_categoria'
, 'causabas_grupo'
, 'causabas_subcategoria'
, 'FLAG_BASE'
            ]
for col in lista_cat:
    print(f'\nPercentual de valores únicos para {col}:')
    print(round((df[col].value_counts()/len(df)) * 100, 2))

Ensino_medio        51.67
Fundamental         24.39
Ensino_superior     21.51
Ignorado             2.04
Sem_escolaridade     0.39
Name: escolaridade_mae, dtype: float64

Percentual de valores únicos para OBITOGRAV:
9.0    0.0
1.0    0.0
2.0    0.0
Name: OBITOGRAV, dtype: float64

Percentual de valores únicos para GRAVIDEZ:
1.0    97.65
2.0     2.17
9.0     0.12
3.0     0.05
Name: GRAVIDEZ, dtype: float64

Percentual de valores únicos para tipo_gravidez:
Unica       97.65
Multipla     2.22
Ignorado     0.13
Name: tipo_gravidez, dtype: float64

Percentual de valores únicos para idade_gestacao_faixa:
entre_37_39    58.65
entre_40_42    27.37
entre_28_36    11.05
Ignorado        2.05
entre_22_27     0.75
menor_22        0.13
Name: idade_gestacao_faixa, dtype: float64

Percentual de valores únicos para SEXO:
1    51.19
2    48.77
0     0.04
Name: SEXO, dtype: float64

Percentual de valores únicos para def_sexo:
Masculino    51.19
Feminino     48.77
Ignorado      0.04
Name: def_sexo, dtype: 

SINASC       98.95
SIM_DOFET     1.05
Name: FLAG_BASE, dtype: float64


### Numerical variables

In [ ]:
# Basic Statistics for Numeric Variables
lista_numerica = [
'IDADEMAE'
, 'SEMAGESTAC'
, 'PESO'
, 'sum_CENTROBS'
, 'sum_QTINST34'
, 'sum_QTINST35'
, 'sum_QTINST36'
, 'sum_QTINST37'
, 'sum_QTLEIT34'
, 'sum_QTLEIT38'
, 'sum_QTLEIT39'
, 'sum_QTLEIT40'
, 'sum_CENTRNEO'
, 'TP_UNID_5'
, 'TP_UNID_7'
, 'TP_UNID_15'
, 'TP_UNID_36'
, 'TP_UNID_61'
]
estatisticas_numericas = df[lista_numerica].describe()
estatisticas_numericas

### Outlier

In [ ]:
def detectar_outliers(col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = col[(col < lower_bound) | (col > upper_bound)]
    return outliers

outliers_numericos = df[lista_numerica].apply(detectar_outliers)

for i in lista_numerica:
    print(outliers_numericos.loc[~outliers_numericos[i].isna(), [i]].describe())

In [ ]:
for i in lista_numerica:
    x1 = df.loc[df['ano_evento'] == 2019, i]
    x2 = df.loc[df['ano_evento'] == 2020, i]
    x3 = df.loc[df['ano_evento'] == 2021, i]
    x4 = df.loc[df['ano_evento'] == 2022, i]

    # Normalize
    kwargs = dict(alpha=0.5, bins=100, density=True, stacked=True)

    # Plot
    plt.figure()  # Creates a new figure for each iteration
    plt.hist(x1, **kwargs, color='g', label='2019')
    plt.hist(x2, **kwargs, color='b', label='2020')
    plt.hist(x3, **kwargs, color='r', label='2021')
    plt.hist(x4, **kwargs, color='y', label='2022')
    plt.gca().set(title=f'Histograma {i}', ylabel='Probability')
    # plt.xlim(50, 75)
    plt.legend()

    # Show or save the figure here if needed
    # plt.savefig(f'histograma_{i}.png')

    plt.show()  # Show the graph

### Cause of death

In [17]:
contingency_table = pd.crosstab(index=df['CAUSABAS'], columns=df['ano_evento'])#, normalize='index')
contingency_table = contingency_table.reset_index()
contingency_table.loc[contingency_table['CAUSABAS'].isin(['P200', 'P95'])]

ano_evento,CAUSABAS,2018,2019,2020,2021,2022
74,P200,2635,2674,1842,1712,1843
141,P95,6094,6172,6634,6701,5793


### Births by municipality per year

In [18]:
df_sinasc = df.loc[df['FLAG_BASE']=='SINASC']
pd.crosstab(index=df_sinasc['evento_MUNNOMEX'], columns=df_sinasc['ano_evento'])

ano_evento,2018,2019,2020,2021,2022
evento_MUNNOMEX,,,,,
ABADIA DE GOIAS,2,5,5,2,6
ABADIA DOS DOURADOS,1,2,0,1,0
ABADIANIA,1,3,5,5,9
ABAETE,218,196,203,164,160
ABAETETUBA,2496,2133,2480,2861,2836
ABAIARA,1,1,2,5,5
ABAIRA,6,4,3,3,4
ABARE,103,96,115,107,88
ABATIA,0,2,0,1,0


# Cleaning data missing the dataset

### Remoção

In [19]:
# Removing columns with a high frequency of missings

#df_limpo = df_limpo.drop(columns=['OBITOGRAV', 'causabas_categoria', 'TIPOBITO', 'causabas_capitulo', 'CAUSABAS'
#                                  , 'def_obito_parto', 'OBITOPARTO', 'causabas_subcategoria', 'causabas_grupo'])

In [6]:
# Removing duplicates SINASC
tam_inicial = len(df)
df_limpo = df.drop_duplicates()
tam_depois=len(df_limpo)
print(f'Remove duplicates {tam_inicial - tam_depois} rows')

Remove duplicates 3908 rows


In [16]:
def count_missing_values(df, columns, group_by):
    """
    Conta a quantidade de valores ausentes para cada coluna especificada, separado por grupos de uma coluna.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame de entrada.
        columns (list): Lista de colunas para contar os valores ausentes.
        group_by (str): Nome da coluna para agrupar os resultados.
        
    Retorna:
        pd.DataFrame: DataFrame com a contagem de valores ausentes por grupo.
    """
    return df.groupby(group_by)[columns].apply(lambda x: x.isnull().sum())

In [20]:
lista_missing = [
    'ESCMAE2010','SEMAGESTAC','PESO','IDADEMAE','GRAVIDEZ', 'res_MUNNOMEX', 'res_CAPITAL', 'res_SIGLA_UF'
    , 'evento_MUNNOMEX', 'evento_CAPITAL', 'evento_REGIAO', 'evento_SIGLA_UF'
]
missing_counts = count_missing_values(df_limpo, lista_missing,  group_by='FLAG_BASE')

In [21]:
print(missing_counts)

           ESCMAE2010  SEMAGESTAC  PESO  IDADEMAE  GRAVIDEZ  res_MUNNOMEX  \
FLAG_BASE                                                                   
SIM_DOFET       13267        8293  8703      9794       581           233   
SINASC         182552      171267  3275       188         0             0   

           res_CAPITAL  res_SIGLA_UF  evento_MUNNOMEX  evento_CAPITAL  \
FLAG_BASE                                                               
SIM_DOFET          233           233              227             227   
SINASC               0             0                0               0   

           evento_REGIAO  evento_SIGLA_UF  
FLAG_BASE                                  
SIM_DOFET            227              227  
SINASC                 0                0  


In [21]:
# Removendo os missing SINASC

#lista_missing_sim = [
#   'sum_CENTROBS','sum_QTINST34','sum_QTINST35','sum_QTINST36','sum_QTINST37','TP_UNID_5'
#    ,'TP_UNID_7','TP_UNID_15','TP_UNID_36','TP_UNID_61', 'res_MUNNOMEX', 'res_CAPITAL', 'res_SIGLA_UF'
#    , 'evento_MUNNOMEX', 'evento_CAPITAL', 'evento_REGIAO', 'evento_SIGLA_UF', 'causabas_capitulo', 'causabas_categoria'
#    , 'causabas_grupo', 'causabas_subcategoria'
#]

lista_missing = [
    'ESCMAE2010','SEMAGESTAC','PESO','IDADEMAE','sum_CENTROBS','sum_QTINST34','sum_QTINST35','sum_QTINST36','sum_QTINST37'
    ,'sum_QTLEIT34','sum_QTLEIT38','sum_QTLEIT39','sum_QTLEIT40','sum_CENTRNEO','TP_UNID_5'
    ,'TP_UNID_7','TP_UNID_15','TP_UNID_36','TP_UNID_61', 'GRAVIDEZ', 'res_MUNNOMEX', 'res_CAPITAL', 'res_SIGLA_UF'
    , 'evento_MUNNOMEX', 'evento_CAPITAL', 'evento_REGIAO', 'evento_SIGLA_UF'
]
tam_antes = len(df_limpo)
#df_limpo_sim = df_limpo_sim.dropna(subset = lista_missing_sim)
df_limpo = df_limpo.dropna(subset = lista_missing)
df_limpo = df_limpo.reset_index(drop=True)
tam_depois=len(df_limpo)
print(f'Remove missing {tam_antes - tam_depois} rows')

Remove missing 364271 rows


In [22]:
# Remove ignored class SINASC
tam_antes = len(df_limpo) 
df_limpo['evento_MUNNOMEX'] = df_limpo['evento_MUNNOMEX'].astype(str).fillna('IGNORADO')
df_limpo = df_limpo.loc[~df_limpo['evento_MUNNOMEX'].str.contains('IGNORADO')]
tam_depois=len(df_limpo)
print(f'Remove ignored evento_MUNNOMEX {tam_antes - tam_depois} rows')

Remove ignored evento_MUNNOMEX 0 rows


In [23]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[~df_limpo['res_MUNNOMEX'].str.contains('IGNORADO')]
tam_depois=len(df_limpo)
print(f'Remove ignored res_MUNNOMEX {tam_antes - tam_depois} rows')

Remove ignored res_MUNNOMEX 487 rows


In [24]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['idademae_faixa']!='Ignorado']
tam_depois=len(df_limpo)
print(f'Remove ignored idademae_faixa {tam_antes - tam_depois} rows')

Remove ignored idademae_faixa 0 rows


In [25]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['ESCMAE2010']!=9]
tam_depois=len(df_limpo)
print(f'Remove ignored ESCMAE2010 {tam_antes - tam_depois} rows')

Remove ignored ESCMAE2010 72739 rows


In [26]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['GRAVIDEZ']!=9]
tam_depois=len(df_limpo)
print(f'Remove ignored GRAVIDEZ {tam_antes - tam_depois} rows')

Remove ignored GRAVIDEZ 6911 rows


In [27]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['idade_gestacao_faixa']!='Ignorado']
tam_depois=len(df_limpo)
print(f'Remove ignored idade_gestacao_faixa {tam_antes - tam_depois} rows')

Remove ignored idade_gestacao_faixa 102524 rows


In [28]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['SEXO']!=0]
tam_depois=len(df_limpo)
print(f'Remove ignored SEXO {tam_antes - tam_depois} rows')

Remove ignored SEXO 4558 rows


In [29]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['peso_faixa']!='Ignorado']
tam_depois=len(df_limpo)
print(f'Remove ignored peso_faixa {tam_antes - tam_depois} rows')

Remove ignored peso_faixa 0 rows


In [30]:
# Mother's age
# Remover somente em SINASC
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[(df_limpo['IDADEMAE']>=10) & (df_limpo['IDADEMAE']<=50)]
tam_depois=len(df_limpo)
print(f'Remove IDADEMAE <10 or >50 {tam_antes - tam_depois} rows')

Remove IDADEMAE <10 or >50 1454 rows


In [31]:
# Weight
std_peso = np.std(df['PESO'])
media_peso = np.mean(df['PESO'])
lim_inf = media_peso - (6 * std_peso)
lim_sup = media_peso + (6 * std_peso)
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['PESO']>0]
df_limpo = df_limpo.loc[(df_limpo['PESO']>=lim_inf) & (df_limpo['PESO']<=lim_sup)]
tam_depois=len(df_limpo)
print(f'Remove PESO 6 std {tam_antes - tam_depois} rows')

Remove PESO 6 std 71 rows


In [32]:
# Weeks of pregnancy less than 22 are not considered deaths, but rather miscarriages. SIM e SINASC
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['idade_gestacao_faixa']!='menor_22']
tam_depois=len(df_limpo)
print(f'Remove idade_gestacao_faixa menor_22 {tam_antes - tam_depois} rows')

Remove idade_gestacao_faixa menor_22 14547 rows


In [33]:
tam_depois = len(df)
print(f'Total remove {tam_inicial - tam_depois} rows')

Total remove 0 rows


In [34]:
df_limpo = df_limpo.reset_index(drop=True)

In [35]:
df_limpo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13337283 entries, 0 to 13337282
Data columns (total 47 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   data_evento            object 
 1   ano_evento             int64  
 2   TIPOBITO               float64
 3   evento_MUNNOMEX        object 
 4   res_MUNNOMEX           object 
 5   evento_CAPITAL         object 
 6   res_CAPITAL            object 
 7   evento_REGIAO          object 
 8   res_REGIAO             object 
 9   evento_SIGLA_UF        object 
 10  res_SIGLA_UF           object 
 11  IDADEMAE               float64
 12  idademae_faixa         object 
 13  ESCMAE2010             float64
 14  escolaridade_mae       object 
 15  OBITOGRAV              float64
 16  GRAVIDEZ               float64
 17  tipo_gravidez          object 
 18  SEMAGESTAC             float64
 19  idade_gestacao_faixa   object 
 20  SEXO                   int64  
 21  def_sexo               object 
 22  PESO            

In [36]:
df_limpo[df_limpo['FLAG_BASE'] == 'SIM_DOFET'].value_counts(['ano_evento'])

ano_evento
2018          20834
2019          19957
2021          19747
2020          19432
2022          18487
dtype: int64

In [37]:
df_limpo.to_csv('base_limpa/base_unificada_limpa_remocao.csv', index=False)